# QA Fetch Matrix V3 Full Probe (Notebook)

Goal:
- Probe every function listed in `docs/05_data_plane/_draft_qa_fetch_rename_matrix_v3.md`
- Classify each function into `pass_has_data / pass_empty / blocked_source_missing / error_runtime`
- Print per-function `head / columns / dtypes / len`
- Export structured artifacts (JSON/CSV + candidate lists)


In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd

repo = Path.cwd()
if str(repo / 'src') not in sys.path:
    sys.path.insert(0, str(repo / 'src'))

from quant_eam.qa_fetch.probe import (
    DEFAULT_EXPECTED_COUNT,
    DEFAULT_MATRIX_V3_PATH,
    probe_matrix_v3,
    results_to_frame,
    write_probe_artifacts,
)

print('repo:', repo)
print('matrix:', DEFAULT_MATRIX_V3_PATH)
print('expected_count:', DEFAULT_EXPECTED_COUNT)


In [ ]:
env_keys = [
    'WEQUANT_MONGO_URI',
    'WEQUANT_DB_NAME',
    'DB_HOST',
    'DB_PORT',
    'DB_USER',
    'DB_NAME',
    'DB_NAME_TEST2',
]
for key in env_keys:
    print(f'{key}={os.getenv(key)}')


In [ ]:
results = probe_matrix_v3(
    matrix_path=DEFAULT_MATRIX_V3_PATH,
    expected_count=DEFAULT_EXPECTED_COUNT,
)
print('total results:', len(results))

df = results_to_frame(results)
display(df[['source','function','status','reason','type','len']])
display(df.groupby(['source','status']).size().rename('count').reset_index())


In [ ]:
import json

for _, row in df.iterrows():
    print('\n===', f"{row['source']}.{row['function']}", '===')
    print('status:', row['status'])
    print('reason:', row['reason'])
    print('type:', row['type'])
    print('len:', row['len'])

    cols = json.loads(row['columns']) if isinstance(row['columns'], str) else row['columns']
    dtypes = json.loads(row['dtypes']) if isinstance(row['dtypes'], str) else row['dtypes']
    head_preview = json.loads(row['head_preview']) if isinstance(row['head_preview'], str) else row['head_preview']

    print('columns:', cols)
    print('dtypes:', dtypes)

    if isinstance(head_preview, list) and len(head_preview) > 0 and isinstance(head_preview[0], dict):
        display(pd.DataFrame(head_preview))
    else:
        print('head_preview:', head_preview)


In [ ]:
artifact_paths = write_probe_artifacts(results, out_dir=repo / 'docs/05_data_plane/qa_fetch_probe_v3')
artifact_paths


In [ ]:
pass_has_data = df[df['status'] == 'pass_has_data'][['source','function','status','len']]
pass_has_data_or_empty = df[df['status'].isin(['pass_has_data','pass_empty'])][['source','function','status','len']]

print('pass_has_data:', len(pass_has_data))
display(pass_has_data)

print('pass_has_data_or_empty:', len(pass_has_data_or_empty))
display(pass_has_data_or_empty)
